# Предобработка данных

In [1]:
import pandas as pd
import numpy as npm

In [4]:
data = pd.read_csv('../data/data_raw.csv')
df = pd.DataFrame(data)

Обработка пропусков

In [8]:
df['TotalCharges'].replace(' ', None, inplace=True)
df['TotalCharges'] = df['TotalCharges'].astype('float')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)